# VacationPy

In [3]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import google_key

### Pulling and Storing the WeatherPy data

In [15]:
# grab the csv file from WeatherPy
path = "../WeatherPy/weather_data.csv"
weather_data = pd.read_csv(path)

# preview the data frame
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,moerai,-22.43,-151.33,69.62,80,22,17.98,PF,1598816030
1,bathsheba,13.22,-59.52,87.80,70,40,12.75,BB,1598815859
2,cape town,-33.93,18.42,48.00,75,0,2.24,ZA,1598816030
3,matagami,49.75,-77.63,60.80,51,75,10.22,CA,1598816030
4,airai,-8.93,125.41,56.39,100,3,4.00,TL,1598816031
...,...,...,...,...,...,...,...,...,...
548,jacareacanga,-6.22,-57.75,79.14,66,99,3.06,BR,1598816180
549,bonga,7.28,36.23,59.92,89,100,1.03,ET,1598816181
550,inirida,3.87,-67.92,74.25,94,100,2.10,CO,1598816181
551,maraa,-1.83,-65.37,87.66,58,97,3.13,BR,1598816181


In [16]:
# Remove any rows missing data
clean_data = weather_data.dropna()
clean_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,moerai,-22.43,-151.33,69.62,80,22,17.98,PF,1598816030
1,bathsheba,13.22,-59.52,87.80,70,40,12.75,BB,1598815859
2,cape town,-33.93,18.42,48.00,75,0,2.24,ZA,1598816030
3,matagami,49.75,-77.63,60.80,51,75,10.22,CA,1598816030
4,airai,-8.93,125.41,56.39,100,3,4.00,TL,1598816031
...,...,...,...,...,...,...,...,...,...
548,jacareacanga,-6.22,-57.75,79.14,66,99,3.06,BR,1598816180
549,bonga,7.28,36.23,59.92,89,100,1.03,ET,1598816181
550,inirida,3.87,-67.92,74.25,94,100,2.10,CO,1598816181
551,maraa,-1.83,-65.37,87.66,58,97,3.13,BR,1598816181


### Humidity Heatmap

In [10]:
# Configure gmaps
gmaps.configure(api_key=google_key)

In [100]:
# Store 'Lat' and 'Lng' into locations
locations = clean_data[['Lat', 'Lng']].astype(float)

# Pull the humidity data as the weight for the heatmap
humidity = clean_data['Humidity'].astype(float)

In [104]:
# Create and add the humidity heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100, point_radius=2)

# Add and view the layer
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

## Conditional DataFrame

In [49]:
# Narrow down the DataFrame to ideal weather conditions
# Ideal Temperatures
good_temp = clean_data[(clean_data["Max Temp"] > 65) & (clean_data["Max Temp"] < 80)]

# Cloudiness
good_clouds = clean_data[(clean_data["Cloudiness"] > 45) & (clean_data["Cloudiness"] < 80)]

# Humidity
good_humid = clean_data[clean_data["Humidity"] < 70]

# merge the conditions to find cities with ideal weather
temp_clouds = pd.merge(good_temp, good_clouds)
ideal_weather = pd.merge(good_humid, temp_clouds)
ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,kutum,14.20,24.67,79.56,56,64,1.59,SD,1598816071
1,palmer,42.16,-72.33,77.00,38,75,11.41,US,1598816084
2,chiredzi,-21.05,31.67,73.11,61,59,8.81,ZW,1598816087
3,los zacatones,22.95,-102.05,69.80,64,75,1.95,MX,1598816109
4,plattsburgh,44.70,-73.45,66.99,48,75,11.41,US,1598815833
5,aksu,41.12,80.26,71.60,39,59,0.74,CN,1598816129
6,roma,43.21,-75.46,71.60,46,75,14.99,US,1598816045
7,atbasar,51.80,68.33,70.29,42,49,5.35,KZ,1598816140
8,waterford,42.70,-83.40,73.00,52,75,5.82,US,1598816143
9,sioux lookout,50.10,-91.92,69.80,52,75,8.05,CA,1598816168


### Hotel Map

In [86]:
# Store the ideal weather cities and add an empty column for hotel names
hotel_df = ideal_weather
hotel_df["Hotel Name"] = ""

In [95]:
# Generate the parameters and base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target_type = "lodging"
radius = 5000

# loop through hotel_df to find the first hotel within 5000m
for index, row in hotel_df.iterrows():
    city = row["City"]
    
    # Generate the cities coordinates as a single string
    lat = row["Lat"]
    lng = row["Lng"]
    coords = str(lat) + ", " + str(lng)
    
    
    # Set the parameters
    params = {
        "location": coords,
        "types": target_type,
        "radius": radius,
        "key": google_key
    }
    
    # Call the data and store in the hotel name column on the hotel_df
    response = requests.get(base_url, params=params)
    hotel_search = response.json()
    
    # Use try/except in case the request failed to find a hotel
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_search['results'][0]['name']
    except:
        hotel_df.loc[index, "Hotel Name"] = "(not found)"
        print(f"{city}: no hotel found nearby.")        

# Preview the update dataframe        
hotel_df        

los zacatones: no hotel found nearby.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,kutum,14.20,24.67,79.56,56,64,1.59,SD,1598816071,Dibbo areaمنطقة دبو
1,palmer,42.16,-72.33,77.00,38,75,11.41,US,1598816084,Wedgewood Motel
2,chiredzi,-21.05,31.67,73.11,61,59,8.81,ZW,1598816087,The Nesbitt Arms
3,los zacatones,22.95,-102.05,69.80,64,75,1.95,MX,1598816109,(not found)
4,plattsburgh,44.70,-73.45,66.99,48,75,11.41,US,1598815833,La Quinta Inn & Suites by Wyndham Plattsburgh
5,aksu,41.12,80.26,71.60,39,59,0.74,CN,1598816129,Pudong Holiday Hotel
6,roma,43.21,-75.46,71.60,46,75,14.99,US,1598816045,Wingate By Wyndham Rome
7,atbasar,51.80,68.33,70.29,42,49,5.35,KZ,1598816140,"The Ritz-Carlton, Astana"
8,waterford,42.70,-83.40,73.00,52,75,5.82,US,1598816143,Olde Mill Inn-Clarkston
9,sioux lookout,50.10,-91.92,69.80,52,75,8.05,CA,1598816168,Knobby's Fly-In Camps


In [105]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [106]:
# Create and add the hotel layer
hotel_markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(hotel_markers)
fig

Figure(layout=FigureLayout(height='420px'))